# importing all libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed, LSTM, Input, BatchNormalization, Conv2D, MaxPooling2D, Reshape, Conv1D, GlobalAveragePooling1D, MaxPooling1D, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.losses import categorical_crossentropy
import tensorflow_hub as hub
from PIL import Image
import gzip
from keras.preprocessing.sequence import pad_sequences
#import spacy
import h5py
import os
import cv2
import pickle
import re
import shutil
import glob
import gzip
%matplotlib inline

In [70]:
import time

# creating path variables

In [4]:
path_gloss_df = '/content/drive/MyDrive/Colab Notebooks/gloss_vector_dataframe.zip'
path_embedding_images = '/content/drive/MyDrive/Colab Notebooks/emb_comp_i3d_train_zero.h5'
path_gloss_files = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.train.annotations_only.gzip'

In [5]:
path_dev_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.dev.annotations_only.gzip'
path_test_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.test.annotations_only.gzip'

In [6]:
path_embedding_dev = '/content/drive/MyDrive/Colab Notebooks/emb_comp_i3d_dev_zero.h5'
path_embedding_test = '/content/drive/MyDrive/Colab Notebooks/emb_comp_i3d_test_zero.h5'

In [7]:
dev_img_embedding = "\\emb_comp_i3d_dev_zero.h5"
test_img_embedding = "\\emb_comp_i3d_test_zero.h5"

# loading annotations in a dataframes

In [8]:
with gzip.open(path_gloss_files,'rb') as f:
  annotation_gloss = pickle.load(f)

In [9]:
with gzip.open(path_dev_gloss,'rb') as f:
  annotation_dev = pickle.load(f)

In [10]:
with gzip.open(path_test_gloss,'rb')as f :
  annotation_test = pickle.load(f)

In [11]:
annotation_gloss = pd.DataFrame(annotation_gloss)

In [12]:
annotation_gloss_dev = pd.DataFrame(annotation_dev)



In [13]:
annotation_gloss_test = pd.DataFrame(annotation_test)

In [14]:
annotation_gloss.head()

,name,signer,gloss,text
0,train/11August_2010_Wednesday_tagesschau-1,Signer08,JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR,und nun die wettervorhersage für morgen donner...
1,train/11August_2010_Wednesday_tagesschau-4,Signer08,ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG ...,mancherorts regnet es auch länger und ergiebig...
2,train/11August_2010_Wednesday_tagesschau-5,Signer08,NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST ...,im nordwesten bleibt es heute nacht meist troc...
3,train/11August_2010_Wednesday_tagesschau-6,Signer08,TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL ...,auch am tag gibt es verbreitet zum teil kräfti...
4,train/11August_2010_Wednesday_tagesschau-7,Signer08,WOLKE LOCH SPEZIELL NORDWEST,größere wolkenlücken finden sich vor allem im ...


In [15]:
annotation_gloss_dev.head()

,name,signer,gloss,text
0,dev/11August_2010_Wednesday_tagesschau-2,Signer08,DRUCK TIEF KOMMEN,tiefer luftdruck bestimmt in den nächsten tage...
1,dev/11August_2010_Wednesday_tagesschau-3,Signer08,ES-BEDEUTET VIEL WOLKE UND KOENNEN REGEN GEWIT...,das bedeutet viele wolken und immer wieder zum...
2,dev/11August_2010_Wednesday_tagesschau-8,Signer08,WIND MAESSIG SCHWACH REGION WENN GEWITTER WIND...,meist weht nur ein schwacher wind aus untersch...
3,dev/25October_2010_Monday_tagesschau-22,Signer01,MITTWOCH REGEN KOENNEN NORDWEST WAHRSCHEINLICH...,am mittwoch hier und da nieselregen in der nor...
4,dev/05May_2011_Thursday_tagesschau-25,Signer08,JETZT WETTER WIE-AUSSEHEN MORGEN FREITAG SECHS...,und nun die wettervorhersage für morgen freita...


In [16]:
annotation_gloss_test.head()

,name,signer,gloss,text
0,test/25October_2010_Monday_tagesschau-17,Signer01,REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KO...,regen und schnee lassen an den alpen in der na...
1,test/25October_2010_Monday_tagesschau-24,Signer01,DONNERSTAG NORDWEST REGEN REGION SONNE WOLKE W...,am donnerstag regen in der nordhälfte in der s...
2,test/15December_2010_Wednesday_tagesschau-37,Signer05,KRAEFTIG AB MORGEN FRUEH MEISTENS SCHNEE SCHNE...,vom nordmeer zieht ein kräftiges tief heran un...
3,test/10March_2011_Thursday_heute-58,Signer01,WOCHENENDE SONNE SAMSTAG SCHOEN TEMPERATUR BIS...,sonnig geht es auch ins wochenende samstag ein...
4,test/14August_2009_Friday_tagesschau-62,Signer05,DEUTSCH LAND MORGEN HOCH DRUCK KOMMEN WOLKE AU...,deutschland liegt morgen unter hochdruckeinflu...


In [17]:
# we can use values attribute to directly create a list of all sentences of a particular column
#annotation_gloss_test["text"].values

# preprocessing the vocabulary

In [18]:
for i in range(len(annotation_gloss)):
  annotation_gloss.iloc[i,2] = 'startseq '+annotation_gloss.iloc[i,2].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = 'startseq '+annotation_gloss.iloc[i,3].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = annotation_gloss.iloc[i,3].replace(' .','')
  #annotation_gloss.iloc[i,3] =annotation_gloss.iloc[i,3][-2].replace(' ','')
 

In [19]:
for i in range(len(annotation_gloss_dev)):
  annotation_gloss_dev.iloc[i,2] = 'startseq '+annotation_gloss_dev.iloc[i,2].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = 'startseq '+annotation_gloss_dev.iloc[i,3].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = annotation_gloss_dev.iloc[i,3].replace(' .','')

In [20]:
for i in range(len(annotation_gloss_test)):
  annotation_gloss_test.iloc[i,2] = 'startseq '+annotation_gloss_test.iloc[i,2].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = 'startseq '+annotation_gloss_test.iloc[i,3].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = annotation_gloss_test.iloc[i,3].replace(' .','')

# creating a vocabulary of texts

In [21]:
vocabulary = []
for txt in annotation_gloss.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_dev.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_dev.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_test.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_test.gloss.values:
    vocabulary.extend(txt.split())
print('Vocabulary Size: %d' % len(set(vocabulary)))

Vocabulary Size: 3606


# reading images embedding for train test and dev

In [22]:
#reading images feature vectors
with h5py.File(path_embedding_images,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_train=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [23]:
#reading images feature vectors
with h5py.File(path_embedding_dev,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_dev=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [24]:
temp_path_dev = "D:\\imp docs\\sem4\\dn\\phoenix t\\PHOENIX-2014-T-release-v3\\PHOENIX-2014-T\\program files"

In [25]:
#reading images feature vectors
with h5py.File(path_embedding_test,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_test=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [26]:
data_dev.shape

(519, 512)

In [27]:
# for i in range(10):
#     for j in range(i+1,10):
#         if(np.array_equal(data_dev,data_dev)):
#             print("true")


# creating tokenizer dataset


In [28]:
gloss_token = []
text_token = []
for i in range(len(annotation_gloss)):
  gloss_token.append(annotation_gloss.iloc[i,2])
  text_token.append(annotation_gloss.iloc[i,3])

In [29]:
gloss_token_dev = []
text_token_dev = []
for i in range(len(annotation_gloss_dev)):
  gloss_token_dev.append(annotation_gloss_dev.iloc[i,2])
  text_token_dev.append(annotation_gloss_dev.iloc[i,3])

In [30]:
gloss_token_test = []
text_token_test = []
for i in range(len(annotation_gloss_test)):
  gloss_token_test.append(annotation_gloss_test.iloc[i,2])
  text_token_test.append(annotation_gloss_test.iloc[i,3])

# creating tokenizer

In [31]:
def create_tokenizer(vocabulary):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=3606,
                                                 oov_token="<unk>",
                                                 filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(vocabulary)
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    return tokenizer

In [32]:
tokenizer = create_tokenizer(vocabulary)

In [33]:
vocab_size = len(tokenizer.word_index)+1
print("vocalb size",vocab_size)

vocalb size 3436


In [34]:
tokenizer.index_word[1]

'<unk>'

In [35]:
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# tokenizing train,test,dev glosses

In [36]:
train_y = encode_sequences(tokenizer,35,gloss_token)
dev_y = encode_sequences(tokenizer,35,gloss_token_dev)
test_y = encode_sequences(tokenizer,35,gloss_token_test)

# hyperparameters

# creating dataset

In [47]:

# def map_func(img_name, cap):
#   temp_data = []
#   cap_data = []
  
#   for i in range(7096):
#     temp_data.append(list(data_train[i]))
#     cap_data.append(list(train_y[i]))
  
#   return temp_data,cap_data

In [48]:
# dataset = tf.data.Dataset.from_tensor_slices((image_name_train, train_y))

# # Use map to load the numpy files in parallel
# dataset = dataset.map(lambda item1, item2: tf.numpy_function(
#           map_func, [item1, item2], [tf.float64, tf.int32]))


In [49]:
#dataset = dataset.batch(BATCH_SIZE)
          
#dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [37]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
num_steps = 7096 // BATCH_SIZE

In [38]:
data_train = tf.cast(data_train,dtype = tf.float32)

In [39]:
def map_func(img_name, cap):
   
   return img_name, cap



In [40]:
dataset = tf.data.Dataset.from_tensor_slices((data_train, train_y))
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [41]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [42]:
for (batch,(img,tar)) in enumerate(dataset.as_numpy_iterator()):
    print(batch)
    print(img)
    print("********")
    print(tar)
    

0
[[ 0.00974285 -0.04300407  0.00038012 ... -0.00592108 -0.00523669
  -0.01416347]
 [ 0.01083652 -0.03858374  0.00077679 ... -0.0010909  -0.00947718
  -0.00609668]
 [ 0.01510646 -0.04167908 -0.00293763 ... -0.0110176  -0.01104077
  -0.01164528]
 ...
 [ 0.01209066 -0.04429029 -0.00156251 ... -0.01190548 -0.00440452
  -0.01498013]
 [ 0.00631018 -0.03350427 -0.00129136 ... -0.00470651 -0.00458341
  -0.00542287]
 [ 0.01310553 -0.04257609 -0.00537116 ... -0.0152493  -0.00465598
  -0.01045003]]
********
[[  2  15 214 ...   0   0   0]
 [  2  38  17 ...   0   0   0]
 [  2 824  20 ...   0   0   0]
 ...
 [  2  12 142 ...   0   0   0]
 [  2  86 176 ...   0   0   0]
 [  2  93  20 ...   0   0   0]]
1
[[ 0.008058   -0.03951006 -0.00316897 ... -0.01185872 -0.0040819
  -0.01630914]
 [ 0.01277918 -0.04673707 -0.00406696 ... -0.00795783 -0.00872108
  -0.0130125 ]
 [ 0.0055579  -0.04028137 -0.00281779 ... -0.01460155 -0.00212066
  -0.01521761]
 ...
 [ 0.01347836 -0.03986605 -0.00613946 ... -0.00600347 -0

### creating vocab embedding

In [155]:
# class VocabularyEmbedder(tf.keras.Model):
    
#     def __init__(self, voc_size, d_model):
#         super(VocabularyEmbedder, self).__init__()
#         self.voc_size = voc_size
#         self.d_model = d_model
#         self.embedder = tf.keras.layers.Embedding(voc_size, d_model)
        
#     def call(self, x): # x - tokens (B, seq_len)
#         x = self.embedder(x)
#         x = x * np.sqrt(self.d_model)
        
#         return x # (B, seq_len, d_model)

In [127]:
#pip install --upgrade tensorflow-lattice

In [159]:

num_layer = 2
d_model = 512
dff = 256
num_heads = 8
row_size = 1

target_vocab_size = len(tokenizer.word_index) + 1
dropout_rate = 0.5

In [160]:
import tensorflow_lattice.layers as tfl

In [206]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    #print("get angel")
    #print("angel shape",angle_rates.shape)
    return pos * angle_rates

In [207]:
def positional_encoding_1d(position, d_model):
    #print("positional encoding 1 dim")
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [208]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [209]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [210]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  #adding -Inf where mask is 1 s.t. value get ignored in softmax

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

# creating feature embedder

In [211]:

# def positional_encoding_2d(row,col,d_model):
#   assert d_model % 2 == 0
#   # first d_model/2 encode row embedding and second d_model/2 encode column embedding
#   row_pos = np.repeat(np.arange(row),col)[:,np.newaxis]
#   col_pos = np.repeat(np.expand_dims(np.arange(col),0),row,axis=0).reshape(-1,1)
#   angle_rads_row = get_angles(row_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)
#   angle_rads_col = get_angles(col_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)
#   #apply sin and cos to odd and even indices resp.
#   angle_rads_row[:, 0::2] = np.sin(angle_rads_row[:, 0::2])
#   angle_rads_row[:, 1::2] = np.cos(angle_rads_row[:, 1::2])
#   angle_rads_col[:, 0::2] = np.sin(angle_rads_col[:, 0::2])
#   angle_rads_col[:, 1::2] = np.cos(angle_rads_col[:, 1::2])
#   pos_encoding = np.concatenate([angle_rads_row,angle_rads_col],axis=1)[np.newaxis, ...]

#   return tf.cast(pos_encoding, dtype=tf.float32)

In [212]:
# class FeatureEmbedder(tf.keras.layers.Layer):
    
#     def __init__(self, d_feat, d_model):
#         super(FeatureEmbedder, self).__init__()
#         self.d_model = d_model
#         print("dodel shape",d_feat)
#         self.embedder = tfl.Linear(d_feat, d_model)
#         print("embedder shape",type(self.embedder))
        
#     def call(self, x): # x - tokens (B, seq_len, d_feat)
#         x = x.reshape()
#         x = self.embedder(x)
#         x = x * np.sqrt(self.d_model)
        
#         return x # (B, seq_len, d_model)

In [213]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask=None):
        #print("multihead attention")
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [214]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [215]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        #print("encoder layer chota")
        

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

In [216]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, 
           look_ahead_mask=None, padding_mask=None):
        #print("decoder layer")
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        # using look ahead mask so that during self attention current query dont consider future token
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        # use padding mask to avoid padded values of both enc_output and dec_input
        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2

In [217]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
               row_size,rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Dense(self.d_model,activation='relu')
        #self.fnc = FeatureEmbedder(d_model,512)

        
        self.pos_encoding = positional_encoding_1d(row_size, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        # shape(x) = (batch_size,seq_len(H*W),features)
        #x = self.fnc(x)
        #print("encoder layer")
        #print("shape of x is ",x.shape)
        seq_len = tf.shape(x)[1]
        #print(tf.shape(x)[1])

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len(H*W), d_model)
        x += self.pos_encoding[:, :seq_len]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [218]:
class Decoder(tf.keras.layers.Layer):
   def __init__(self, num_layers,d_model,num_heads,dff, target_vocab_size, maximum_position_encoding,   rate=0.1):
      super(Decoder, self).__init__()
      self.d_model = d_model
      self.num_layers = num_layers

      self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
      self.pos_encoding = positional_encoding_1d(maximum_position_encoding, d_model)

      self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]
      self.dropout = tf.keras.layers.Dropout(rate)

   def call(self, x, enc_output, training,look_ahead_mask=None, padding_mask=None):
      seq_len = tf.shape(x)[1]
      attention_weights = {}
      #print("shape of seq_len ",seq_len)
      #print("x shape in decoder cakk",x.shape)

      x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
      x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
      x += self.pos_encoding[:, :seq_len, :]
      x = self.dropout(x, training=training)

      for i in range(self.num_layers):
         x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                            look_ahead_mask, padding_mask)
         
         attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
         attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

      return x, attention_weights

In [219]:
class Transformer(tf.keras.Model):
   def __init__(self, num_layers, d_model, num_heads, dff,row_size,
              target_vocab_size,max_pos_encoding, rate=0.1,use_linear_embedder =True):
      super(Transformer, self).__init__()
       
      self.encoder = Encoder(num_layers, d_model, num_heads, dff,row_size, rate)
      self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                          target_vocab_size,max_pos_encoding, rate)
      self.final_layer = tf.keras.layers.Dense(target_vocab_size)

   def call(self, inp, tar, training,look_ahead_mask=None,dec_padding_mask=None,enc_padding_mask=None ):
      # print("inp.shape",inp.shape)
      # print("training shape",training.shape)
      # print("enc_padding_mask ",enc_padding_mask.shape)
      enc_output = self.encoder(inp, training)  # (batch_size, inp_seq_len, d_model      )
      dec_output, attention_weights = self.decoder(
      tar, enc_output, training, look_ahead_mask, dec_padding_mask)
      final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
      return final_output, attention_weights

In [220]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        #print("custom schedule")
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [221]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [222]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [223]:

def loss_function(real, pred):
    #print("loss_function")
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [224]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [225]:
transformer = Transformer(num_layer,d_model,num_heads,dff,row_size,target_vocab_size,max_pos_encoding=target_vocab_size,rate=dropout_rate)

In [226]:

def create_masks_decoder(tar):
    #print("msak decoder run")
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    return combined_mask

In [227]:
@tf.function
def train_step(img_tensor, tar):
   #print("train run")
   tar_inp = tar[:, :-1]
   #print(tar_inp)
   
   tar_real = tar[:, 1:]
  # print(tar_real)
   dec_mask = create_masks_decoder(tar_inp)
   #print("j9j")
   with tf.GradientTape() as tape:
      predictions, _ = transformer(img_tensor, tar_inp,True, dec_mask)
      loss = loss_function(tar_real, predictions)

   gradients = tape.gradient(loss, transformer.trainable_variables)   
   optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
   train_loss(loss)
   train_accuracy(tar_real, predictions)

In [228]:
for epoch in range(30):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
  
    for (batch, (img_tensor, tar)) in enumerate(dataset.as_numpy_iterator()):
        #print("eopch run")
        train_step(img_tensor, tar)

        if batch % 50 == 0:
            

            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result()))


    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 8.3076 Accuracy 0.0000
Epoch 1 Batch 50 Loss 8.2057 Accuracy 0.0003
Epoch 1 Batch 100 Loss 7.9594 Accuracy 0.0103
Epoch 1 Loss 7.8973 Accuracy 0.0120
Time taken for 1 epoch: 11.758482694625854 secs

Epoch 2 Batch 0 Loss 7.2263 Accuracy 0.0294
Epoch 2 Batch 50 Loss 6.8381 Accuracy 0.0294
Epoch 2 Batch 100 Loss 6.4849 Accuracy 0.0294
Epoch 2 Loss 6.4205 Accuracy 0.0294
Time taken for 1 epoch: 5.6277453899383545 secs

Epoch 3 Batch 0 Loss 5.6805 Accuracy 0.0294
Epoch 3 Batch 50 Loss 5.4789 Accuracy 0.0294
Epoch 3 Batch 100 Loss 5.3420 Accuracy 0.0294
Epoch 3 Loss 5.3177 Accuracy 0.0294
Time taken for 1 epoch: 5.497990369796753 secs

Epoch 4 Batch 0 Loss 5.2250 Accuracy 0.0294
Epoch 4 Batch 50 Loss 5.0945 Accuracy 0.0294
Epoch 4 Batch 100 Loss 5.0475 Accuracy 0.0300
Epoch 4 Loss 5.0366 Accuracy 0.0301
Time taken for 1 epoch: 5.452704668045044 secs

Epoch 5 Batch 0 Loss 5.0587 Accuracy 0.0312
Epoch 5 Batch 50 Loss 4.9508 Accuracy 0.0322
Epoch 5 Batch 100 Loss 4.9371 Acc

 # loss

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
   from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# training

In [ ]:
temp = tf.expand_dims([tokenizer.word_index['startseq']] * BATCH_SIZE, 1)

In [ ]:
temp.shape

In [ ]:
loss_plot = []

@tf.function
def train_step(img_tensor, target):
 loss = 0
 # initializing the hidden state for each batch
 # because the captions are not related from image to image

 # hidden shape = (128,1024)
 hidden = decoder.reset_state(batch_size=target.shape[0])
 dec_input = tf.expand_dims([tokenizer.word_index['startseq']] * BATCH_SIZE, 1)

 with tf.GradientTape() as tape:
     features = img_tensor
     for i in range(1, target.shape[1]):
         # passing the features through the decoder
         predictions, hidden, _ = decoder(dec_input, features, hidden)
         loss += loss_function(target[:, i], predictions)

         # using teacher forcing
         dec_input = tf.expand_dims(target[:, i], 1)

 total_loss = (loss / int(target.shape[1]))
 trainable_variables = decoder.trainable_variables
 gradients = tape.gradient(loss,trainable_variables)
 optimizer.apply_gradients(zip(gradients, trainable_variables))

 return loss, total_loss

In [ ]:

EPOCHS = 10
with tf.device('/gpu:0'):
    for epoch in range(0, EPOCHS):
       start = time.time()
       total_loss = 0

       for (batch, (img_tensor, target)) in enumerate(dataset):
           batch_loss, t_loss = train_step(img_tensor, target)
           total_loss += t_loss

           if batch % 100 == 0:
               print ('Epoch {} Batch {} Loss {:.4f}'.format(
                 epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
       # storing the epoch end loss value to plot later
       loss_plot.append(total_loss / num_steps)

       print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                            total_loss/num_steps))

       print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

# evaluate using greedy

In [ ]:
def evaluate(image):
   attention_plot = np.zeros((max_length, attention_features_shape))

   hidden = decoder.reset_state(batch_size=1)
   temp_input = tf.expand_dims(load_image(image)[0], 0)
   img_tensor_val = image_features_extract_model(temp_input)
   img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3])

   features = encoder(img_tensor_val)
   dec_input = tf.expand_dims([tokenizer.word_index['startseq']], 0)
   result = []

   for i in range(max_length):
       predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
       attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
       predicted_id = tf.argmax(predictions[0]).numpy()
       result.append(tokenizer.index_word[predicted_id])

       if tokenizer.index_word[predicted_id] == 'endseq':
           return result, attention_plot

       dec_input = tf.expand_dims([predicted_id], 0)
   attention_plot = attention_plot[:len(result), :]

   return result, attention_plot

In [ ]:
# def plot_attention(image, result, attention_plot):
#    temp_image = np.array(Image.open(image))
#    fig = plt.figure(figsize=(10, 10))
#    len_result = len(result)
#    for l in range(len_result):
#        temp_att = np.resize(attention_plot[l], (8, 8))
#        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
#        ax.set_title(result[l])
#        img = ax.imshow(temp_image)
#        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

#    plt.tight_layout()
#    plt.show()

In [ ]:
# captions on the validation set
image = data_test[0]

# real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

# remove <start> and <end> from the real_caption
first = real_caption.split(' ', 1)[1]
real_caption = 'Two white dogs are playing in the snow'

#remove "<unk>" in result
for i in result:
   if i=="<unk>":
       result.remove(i)

for i in real_caption:
   if i=="<unk>":
       real_caption.remove(i)

#remove <end> from result        
result_join = ' '.join(result)
result_final = result_join.rsplit(' ', 1)[0]

real_appn = []
real_appn.append(real_caption.split())
reference = real_appn
candidate = result

score = sentence_bleu(reference, candidate)
print(f"BELU score: {score*100}")

print ('Real Caption:', real_caption)
print ('Prediction Caption:', result_final)
plot_attention(image, result, attention_plot)

In [ ]:
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
start = time.time()
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

first = real_caption.split(' ', 1)[1]
real_caption = first.rsplit(' ', 1)[0]

#remove "<unk>" in result
for i in result:
   if i=="<unk>":
       result.remove(i)

#remove <end> from result        
result_join = ' '.join(result)
result_final = result_join.rsplit(' ', 1)[0]

real_appn = []
real_appn.append(real_caption.split())
reference = real_appn
candidate = result_final

print ('Real Caption:', real_caption)
print ('Prediction Caption:', result_final)

plot_attention(image, result, attention_plot)
print(f"time took to Predict: {round(time.time()-start)} sec")

Image.open(img_name_val[rid])